## Labeling using regex groups

In [1]:
from pathlib import Path
import pandas as pd

# load full_df if not loaded
DATA_PATH = '../competitors-xgb/data'
file_path = (
    Path(DATA_PATH) / 'tables_OZ_geo_5500' /
    'processed' / 'OZ_geo_5500.csv'
)
full_df = pd.read_csv(file_path) \
    .drop(columns='sales.1') # TODO: not include this column when creating the dataframe in make_OZ_geo_5500

# combine 'name' and 'description'
full_df.description = full_df.description.fillna('')
full_df.name.isna().sum(), full_df.description.isna().sum()
full_df['name_and_description'] = full_df['name'] + '.\n' + full_df['description']
pd.set_option('display.max_colwidth', None)
full_df[['sku', 'name_and_description']].head(1)

,sku,name_and_description
0,1871769771,"Карты МИРА и РОССИИ настенные политические,160х102 см, Комплект школьных карт для детей / 2024 г.\nПредставляем вашему вниманию уникальный набор карт, состоящий из Географической карты России и Настенной карты мира. Каждая карта имеет размер 102х160 см и масштаб 1:5 500 000 для России и 1:22 000 000 для мира. Эти современные издания 2024 года отражают актуальные изменения с новыми границами и помогут вам в образовательных целях, декоре и повседневной жизни.\nГеографическая карта России — идеальный помощник для школьников и любителей географии. Она прекрасно иллюстрирует все регионы страны, их особенности и природные зоны. Настенная карта России характеризуется четкой и детализированной графикой, что делает её удобной для изучения. Эта физическая карта позволяет лучше понять разнообразие природных зон России и узнать о местной флоре и фауне. Яркие цвета и четкие обозначения обеспечивают высокую читаемость, что позволяет легко находить нужные области и изучать их.\nНастенная карта мира станет отличным выбором для образовательных целей и домашнего использования. Она помогает изучать страны, их границы и столицы, развивая пространственное мышление. Политическая карта мира выполнена в ярких цветах и с четкими границами, включая моря, океаны и флаги стран. Она подойдет как для школьных кабинетов, так и для стильного оформления интерьера вашего дома.\nОбъединяя два товара, совместите полезное с эстетическим — карты можно использовать как современные учебные пособия для школьников и взрослых, а также как элементы декора. Этот набор станет прекрасным подарком на 1 сентября, Новый год и другие праздники. Ламинированная поверхность защищает карты от повреждений, делая их долговечными и удобными для использования.\nВыбирая набор карт, вы обеспечиваете полезный и функциональный инструмент для изучения нашей страны и мира, что делает их незаменимыми в учебном процессе и повседневной жизни."


In [2]:
# Extract size

import pandas as pd
import numpy as np

# Regex pattern: match width x height with optional unit (мм, см, м)
size_pattern = r'(\d+(?:[.,]\d+)?)\s*[xхХ×\*]\s*(\d+(?:[.,]\d+)?)(?:\s*(мм|см|м))?'

# Extract all size_matches_df from 'name_and_description'
size_matches_df = full_df['name_and_description'].str.extractall(size_pattern)

# Normalize numbers (comma to dot) and convert to float
size_matches_df[0] = size_matches_df[0].str.replace(',', '.', regex=False).astype(float)
size_matches_df[1] = size_matches_df[1].str.replace(',', '.', regex=False).astype(float)

# Standardize unit column
size_matches_df[2] = size_matches_df[2].str.lower().fillna('см')  # default to cm

# Rename columns
size_matches_df.columns = ['width_raw', 'height_raw', 'unit_raw']

# Bigger size goes first -> allow symmetrical sizes (100x60 = 60x100)
size_matches_df[["width_raw","height_raw"]] = np.sort(size_matches_df[["width_raw","height_raw"]].values, axis=1)[:,::-1]

# Convert to centimeters
conversion_factors = {'мм': 0.1, 'см': 1, 'м': 100}
size_matches_df['factor'] = size_matches_df['unit_raw'].map(conversion_factors)
size_matches_df['width'] = size_matches_df['width_raw'] * size_matches_df['factor']
size_matches_df['height'] = size_matches_df['height_raw'] * size_matches_df['factor']
size_matches_df = size_matches_df.astype({'width': int, 'height': int})

# Add SKU info
size_matches_df = size_matches_df.reset_index().merge(full_df[['sku']], left_on='level_0', right_index=True)

# Final output
size_matches_df = size_matches_df[['sku', 'width', 'height', 'unit_raw', 'width_raw', 'height_raw']]

In [3]:
# View extracted sizes 
print(f'SKU with extracted sizes: {size_matches_df.sku.nunique()}')

nosize_sku = set(full_df['sku'].tolist()) - set(size_matches_df['sku'].tolist())
nosize_sku = list(nosize_sku)
print(f'SKU without extracted sizes: {len(nosize_sku)}')

pd.reset_option('display.max_rows')
display(size_matches_df)

SKU with extracted sizes: 4957
SKU without extracted sizes: 605


,sku,width,height,unit_raw,width_raw,height_raw
0,1871769771,160,102,см,160.0,102.0
1,1871769771,160,102,см,160.0,102.0
2,1679550303,70,50,м,0.7,0.5
3,1679550303,70,50,м,0.7,0.5
4,1679550303,70,50,см,70.0,50.0
...,...,...,...,...,...,...
7914,166584090,122,79,см,122.0,79.0
7915,166451882,60,40,см,60.0,40.0
7916,154409524,83,39,см,83.0,39.3
7917,147896031,80,60,см,80.0,60.0


In [16]:
# Define regex pattern dictionary per each sku

ALL_PATTERNS_DICT = {
    # DONE
    1871769771: {
        True: {
            "мира",
            "россии",
            "настенн",
            "политическ",
            r"комплект школьных карт|карта двух?сторонняя|двух?сторонняя(?:\s+\w+){0,5}\s+карта|\b(?:[2-9]|\d{2,})\s*шт(?:\.|ук|уки)?\b",
            # r"|\b(?:[2-9]|\d{2,})\s*шт(?:\.|ук|уки)?\b"
        },
        False: {
            # 'тубус',
            # "отвес|рейк",
            'физическ',
            # r'административная',
            'фотообои',
            'скретч',
            'английск',
            "ретро|старинн",
            'полушар',
            "фальцован|складн",
        }
    },

    # # DONE (no positives in dataset)
    # 1679550303: {
    #     True: {
    #         "москв",
    #         "метро",
    #     },
    #     False: {
    #         "фальцованая|складная",
    #         "настенн"
    #     }
    # },

    # DONE
    # False Positives: 
    1200553001: {
        True: {
            "политическая",
            "мира",
            # "тубус",
        },
        False: {
            # 'отвес|рейк',
            r"комплект школьных карт|карта двух?сторонняя|двух?сторонняя(?:\s+\w+){0,5}\s+карта|\b(?:[2-9]|\d{2,})\s*шт(?:\.|ук|уки)?\b",

            'карта россии',
            'физическ',
            'административная',
            'фотообои',
            'скретч',
            'английск',
            "ретро|старинн",
            'полушар',
            "фальцован|складн",
        }
    },

    # DONE
    922231521: {
        True: {
            "политическ",
            "мира",
            # "тубус",
        },
        False: {
            # 'отвес|рейк', # TODO: 'можно заказать карту в комплекте с пластиковыми отвесами' -> False Negative
            r"комплект школьных карт|карта двух?сторонняя|двух?сторонняя(?:\s+\w+){0,5}\s+карта|\b(?:[2-9]|\d{2,})\s*шт(?:\.|ук|уки)?\b",

            'карта россии',
            'физическ',
            'административная',
            'фотообои',
            'скретч',
            'английск',
            "ретро|старинн",
            'полушар',
            "фальцован|складн",
        }
    },

    # DONE
    # False Positives: 
    # - 1438364140: выбор карты 99x160 или 100x60 (101x60)
    922230517: {
        True: {
            "политическ",
            "мира",
            "настенн",
            # "тубус",
        },
        False: {
            # 'отвес|рейк', # TODO: 'можно заказать карту в комплекте с пластиковыми отвесами' -> False Negative
            r"комплект школьных карт|карта двух?сторонняя|двух?сторонняя(?:\s+\w+){0,5}\s+карта|\b(?:[2-9]|\d{2,})\s*шт(?:\.|ук|уки)?\b",

            'карта россии',
            'физическ',
            'административная',
            'фотообои',
            'скретч',
            'английск',
            "ретро|старинн",
            'полушар',
            "фальцован|складн",
        }
    },

    # DONE
    # False Positives: 
    # - 166584091: описание нескольких карт в ассортименте (в т.ч. скретч-карта)
    922230183: {
        True: {
            # "географическая|по географии",
            "политическая|политико-административная|административно-политическая",
            "карта россии",
            "тубус",
        },
        False: {
            # 'отвес|рейк', # TODO: 'можно заказать карту в комплекте с пластиковыми отвесами' -> False Negative
            r"комплект школьных карт|карта двух?сторонняя|двух?сторонняя(?:\s+\w+){0,5}\s+карта|\b(?:[2-9]|\d{2,})\s*шт(?:\.|ук|уки)?\b",
            r'\b(?:[2-9]|\d{2,})\s*шт(?:\.|ук|уки)?\b',

            'английск',
            # 'физическ',
            'фотообои',
            'скретч',
            "ретро|старинн",
            'полушар',
            "фальцован|складн",
        }
    },

    # DONE
    922229770: {
        True: {
            # "географическая|по географии",
            "политическая|политико-административная|административно-политическая",
            "карта россии",
            # "тубус",
        },
        False: {
            # 'отвес|рейк', # TODO: 'можно заказать карту в комплекте с пластиковыми отвесами' -> False Negative
            r"комплект школьных карт|карта двух?сторонняя|двух?сторонняя(?:\s+\w+){0,5}\s+карта|\b(?:[2-9]|\d{2,})\s*шт(?:\.|ук|уки)?\b",

            'детская',

            'английск',
            # 'физическ',
            'фотообои',
            'скретч',
            "ретро|старинн",
            'полушар',
            "фальцован|складн",
        }
    },

    # DONE
    824158517: {
        True: {
            # "географическая|по географии",
            "политическая",
            'мир',
            # "тубус",
        },
        False: {
            # 'отвес|рейк', # TODO: 'можно заказать карту в комплекте с пластиковыми отвесами' -> False Negative
            r"комплект школьных карт|двух?сторонняя карта",

            r"(?:\s+(?!ламинация\b)[\w-]+(?:\s*[,.:;]\s*)?)"  # ← allow an optional , . : ; between the word and target
            r"\s+двух?сторонняя|стороны\b"                            # “двусторонняя” / “двухсторонняя”
            r"(?!\s+ламинация\b)",                            # not immediately followed by “ламинация”

            r'\b(?:[2-9]|\d{2,})\s*шт(?:\.|ук|уки)?\b',

            'политико-административная|административно-политическая',
            "карта россии",
            'иллюстрированн|с иллюстрациями',

            'английск',
            # 'физическ',
            'фотообои',
            'скретч',
            "ретро|старинн",
            'полушар',
            "фальцован|складн",
        }
    },

    # DONE
    508612558: {
        True: {
            "москв",
            "настен",
            "метро",
            "с линиями метро"
        },
    },


    # DONE
    508611672: {
        True: {
            'москв',
            'московск.*област',
        },
        False: {
            'округ',
            'Большая Москва', # HACK: too local
        }
    },

    # DONE
    507113963: {
        True: {
            'настенн',
            'европы',
            'политическ',
        },
    },

    # DONE
    # False Negatives:
    # - не указаны размеры, по хактеристикам подходит: 1538773969, 1043438100, 1126764501, 1129697481, 1215475736, 1573135174
    494562010: {
        True: {
            "мира? и|и мира?",
            "росси",
            "складн|фальцован",
            "политическ",
            # 'политико-административная|административно-политическая',
        },
        False: {
            # "отвес|рейк",
            # r"комплект школьных карт|карта двух?сторонняя|двух?сторонняя карта|шт[\.|ук|уки]",

            'физическ',
            'фотообои',
            'скретч',
            'английск',
            "ретро|старинн",
            'полушар',
        }
    },

    # # DONE (no positives in dataset)
    # 492260072: {
    #     True: {
    #         "(?:атлас|карта) автомобильных дорог росси", # False Negative: 1710867998 - указано 'карта автомобильных дорог России' в карте Москвы (на сайте уже нет)
    #     },
    # },

    # DONE
    491279127: {
        True: {
            "политическ",
            "мира",
            # "тубус",
        },
        False: {
            # 'отвес|рейк', # TODO: 'можно заказать карту в комплекте с пластиковыми отвесами' -> False Negative
            r"комплект школьных карт|карта двух?сторонняя|двух?сторонняя(?:\s+\w+){0,5}\s+карта|\b(?:[2-9]|\d{2,})\s*шт(?:\.|ук|уки)?\b",

            'карта россии',
            'физическ',
            'административная',
            'фотообои',
            'скретч',
            'английск',
            "ретро|старинн",
            'полушар',
            "фальцован|складн",
            'контурная',
            'историческая',
        }
    },

    # DONE
    491273791: {
        True: {
            "россии",
            "настенн",
            "физическая",
        },
        False: {
            r"комплект школьных карт|карта двух?сторонняя|двух?сторонняя(?:\s+\w+){0,5}\s+карта|\b(?:[2-9]|\d{2,})\s*шт(?:\.|ук|уки)?\b",
            "политич",
            'административн',
            'природные зоны',
            'фотообои',
            'скретч',
            'английск',
            "ретро|старинн",
            'полушар',
            "фальцован|складн",
            'контурная',
            'историческая',
        },
    },

    # DONE
    491273438: {
        True: {
            "росси",
            # "настенн",
            # "физическая",
            'природн.*зон',
        },
        False: {
            r"комплект школьных карт|карта двух?сторонняя|двух?сторонняя(?:\s+\w+){0,5}\s+карта|\b(?:[2-9]|\d{2,})\s*шт(?:\.|ук|уки)?\b",
            "политич",
            'административн',
            'фотообои',
            'скретч',
            'английск',
            "ретро|старинн",
            'полушар',
            "фальцован|складн",
            'контурная',
            'историческая',
        },

    },

    # DONE
    491271768: {
        True: {
            "москв",
            # "путеводитель",
            "фальцован|складн",
        },
        False: {
            # r"комплект школьных карт|карта двух?сторонняя|двух?сторонняя(?:\s+\w+){0,5}\s+карта|\b(?:[2-9]|\d{2,})\s*шт(?:\.|ук|уки)?\b",
            r"\b(?:[2-9]|\d{2,})\s*шт(?:\.|ук|уки)?\b",
            'атлас автомобильных дорог',
            'английск',
            'черного моря',
            'две карты',
        },
        'include': [
            1688947923 # 'атлас автомобильных дорог'
        ]
    },

    # # DONE (no positives in dataset)
    # 491271339: {
    #     True: {
    #         "москв",
    #         # "путеводитель",
    #         "фальцован|складн",
    #         "англ",
    #     },
    #     False: {
    #         # r"комплект школьных карт|карта двух?сторонняя|двух?сторонняя(?:\s+\w+){0,5}\s+карта|\b(?:[2-9]|\d{2,})\s*шт(?:\.|ук|уки)?\b",
    #         r"\b(?:[2-9]|\d{2,})\s*шт(?:\.|ук|уки)?\b",
    #         'атлас автомобильных дорог',
    #         'черного моря',
    #         'две карты',
    #     },
    # },

    # DONE
    491271320: {
        True: {
            # "географическая|по географии",
            "политическая|политико-административная|административно-политическая",
            "карта россии",
            "тубус",
        },
        False: {
            # 'отвес|рейк', # TODO: 'можно заказать карту в комплекте с пластиковыми отвесами' -> False Negative
            r"комплект школьных карт|карта двух?сторонняя|двух?сторонняя(?:\s+\w+){0,5}\s+карта|\b(?:[2-9]|\d{2,})\s*шт(?:\.|ук|уки)?\b",
            r'\b(?:[2-9]|\d{2,})\s*шт(?:\.|ук|уки)?\b',

            'английск',
            # 'физическ',
            'фотообои',
            'скретч',
            "ретро|старинн",
            'полушар',
            "фальцован|складн",
        }
    },

    # DONE
    491271284: {
        True: {
            'физическ',
            'мир',
            'настенн',
        },
        False: {
            r"комплект школьных карт|карта двух?сторонняя|двух?сторонняя(?:\s+\w+){0,5}\s+карта|\b(?:[2-9]|\d{2,})\s*шт(?:\.|ук|уки)?\b",
            r'\b(?:[2-9]|\d{2,})\s*шт(?:\.|ук|уки)?\b',
            
            'россии',
            
            'английск',
            'политическая',
            'фотообои',
            'скретч',
            "ретро|старинн",
            'полушар',
            "фальцован|складн",
            'иллюстрированн|с иллюстрациями|с животными',
        }
    },

    # DONE
    491270369: {
        True: {
            "политическая|политико-административная|административно-политическая",
            "карта россии",
        },
        False: {
            # "тубус",
            # 'отвес|рейк', # TODO: 'можно заказать карту в комплекте с пластиковыми отвесами' -> False Negative
            r"комплект школьных карт|карта двух?сторонняя|двух?сторонняя(?:\s+\w+){0,5}\s+карта|\b(?:[2-9]|\d{2,})\s*шт(?:\.|ук|уки)?\b",
            r'\b(?:[2-9]|\d{2,})\s*шт(?:\.|ук|уки)?\b',

            'английск',
            # 'физическ',
            'фотообои',
            'скретч',
            "ретро|старинн",
            'полушар',
            "фальцован|складн",
        }
    },

    # DONE
    491270272: {
        True: {
            "политическ",
            "мир",
            # "тубус",
            "ретро|старинн",
        },
        False: {
            # 'отвес|рейк', # TODO: 'можно заказать карту в комплекте с пластиковыми отвесами' -> False Negative
            r"комплект школьных карт|карта двух?сторонняя|двух?сторонняя(?:\s+\w+){0,5}\s+карта|\b(?:[2-9]|\d{2,})\s*шт(?:\.|ук|уки)?\b",
            'карта россии',
            'физическ',
            'административная',
            'фотообои',
            'скретч',
            'английск',
            'полушар',
            "фальцован|складн",
        }
    },

    # DONE
    491268805: {
        True: {
            "москв",
            # "метро",
            'настенн',
        },
        False: {
            "фальцован|складн",
            'репринт'            
        }
    },
}

In [17]:
# CHOSEN_SKU = 1871769771
# CHOSEN_SKU = 1200553001
# CHOSEN_SKU = 491271768
# CHOSEN_SKU = 508611672
# CHOSEN_SKU = 508612558
# CHOSEN_SKU = 491271768
# CHOSEN_SKU = 491271339
# CHOSEN_SKU = 491271320
# CHOSEN_SKU = 491271284
# CHOSEN_SKU = 491270369
# CHOSEN_SKU = 491270272
# CHOSEN_SKU = 491268805
CHOSEN_SKU = 1871769771

chosen_subset_df = full_df

COLUMN_TO_MATCH = 'name_and_description'

MARGIN = 0.20

SHOW_ALL_ROWS = False
# SHOW_ALL_ROWS = True

In [18]:
# process_sku_matches

import operator
from functools import reduce
import pandas as pd

def process_sku_matches(
    chosen_sku: str,
    pattern_dict: dict,
    size_matches_df: pd.DataFrame,
    chosen_subset_df: pd.DataFrame,
    column_to_match: str,
    margin: float
):
    # 1) unpack the patterns for this SKU
    pos_patterns = pattern_dict.get(True,  [])
    neg_patterns = pattern_dict.get(False, [])

    # 2) build query_row with its true size
    query_size_df = size_matches_df[size_matches_df["sku"] == chosen_sku]
    q_w, q_h      = query_size_df.iloc[0][["width", "height"]]
    query_row     = chosen_subset_df[chosen_subset_df['sku'] == chosen_sku].copy()
    query_row['width'], query_row['height'] = q_w, q_h

    # 3) build a working regex‐flag DataFrame
    chosen_regex_df = chosen_subset_df.copy()
    for p in pos_patterns:
        col = f"P: {p}"
        chosen_regex_df[col] = chosen_regex_df[column_to_match]\
            .str.contains(p, case=False, na=False)
        query_row[col]       = query_row[column_to_match]\
            .str.contains(p, case=False, na=False)
    for n in neg_patterns:
        col = f"N: {n}"
        chosen_regex_df[col] = chosen_regex_df[column_to_match]\
            .str.contains(n, case=False, na=False)
        query_row[col]       = query_row[column_to_match]\
            .str.contains(n, case=False, na=False)

    # 4) pattern columns
    pattern_cols = [f"P: {p}" for p in pos_patterns] + [f"N: {n}" for n in neg_patterns]

    # 5) build masks
    pos_masks = [chosen_regex_df[c] for c in pattern_cols if c.startswith("P: ")]
    neg_masks = [~chosen_regex_df[c] for c in pattern_cols if c.startswith("N: ")]
    base_index = chosen_regex_df.index

    combined_mask = (
        reduce(operator.and_, pos_masks + neg_masks)
        if (pos_masks + neg_masks) else pd.Series(True, index=base_index)
    )
    pos_mask = (
        reduce(operator.and_, pos_masks)
        if pos_masks else pd.Series(True, index=base_index)
    )

    rejected  = chosen_regex_df.loc[~pos_mask   & (chosen_regex_df["sku"] != chosen_sku)].copy()
    positives = chosen_regex_df.loc[ combined_mask & (chosen_regex_df["sku"] != chosen_sku)]

    # 6) size‐matching helpers
    def best_match(col, sku):
        q_vals = size_matches_df.loc[size_matches_df["sku"] == chosen_sku, col].unique()
        c_vals = size_matches_df.loc[size_matches_df["sku"] == sku, col].unique()
        if not len(q_vals) or not len(c_vals):
            return None, None, float("inf")
        return min(
            ((c, q, abs(c - q) / q) for c in c_vals for q in q_vals),
            key=lambda x: x[2]
        )

    def add_size_cols(df, col):
        out = df.copy()
        triples = out["sku"].apply(lambda s: best_match(col, s))
        out[f"{col}_matched_cand"]  = triples.apply(lambda t: t[0])
        out[f"{col}_matched_query"] = triples.apply(lambda t: t[1])
        out[f"{col}_delta"]         = triples.apply(lambda t: t[2])
        return out

    def filter_by(col, df, inverse=False):
        tmp = add_size_cols(df, col)
        m   = tmp[f"{col}_delta"] <= margin
        return tmp[~m] if inverse else tmp[m]

    # 7) gather hard_negatives
    if neg_masks:
        neg_fail_mask = pos_mask & ~reduce(operator.and_, neg_masks)
        pattern_negs  = chosen_regex_df.loc[
            neg_fail_mask & (chosen_regex_df["sku"] != chosen_sku)
        ].copy()
    else:
        pattern_negs = pd.DataFrame(columns=chosen_regex_df.columns)

    neg_by_w = filter_by("width",  positives, inverse=True)
    neg_by_h = filter_by("height", positives, inverse=True)

    pattern_negs['fail_regex'] = True
    pattern_negs['fail_size']  = False

    neg_by_size = pd.concat([neg_by_w, neg_by_h])
    neg_by_size['fail_size']   = True

    hard_negatives = (
        pd.concat([pattern_negs, neg_by_size])
        .drop_duplicates(subset="sku")
        .pipe(add_size_cols, col="width")
        .pipe(add_size_cols, col="height")
    )
    hard_negatives['fail_regex'] = hard_negatives['fail_regex'].astype('boolean').fillna(False)

    # 8) re‐filter positives by size
    positives = filter_by("width",  positives)
    positives = filter_by("height", positives)

    # 9) assemble display_cols
    size_cols = [
        "width_matched_cand","width_matched_query","width_delta",
        "height_matched_cand","height_matched_query","height_delta"
    ]
    fail_cols = ["fail_regex","fail_size"]

    return positives, hard_negatives, rejected, query_row, fail_cols, size_cols, pattern_cols

In [19]:
# print_sku_matches

def print_sku_matches(
    positives: pd.DataFrame,
    hard_negatives: pd.DataFrame,
    rejected: pd.DataFrame,
    query_row: pd.DataFrame,
    display_cols: list,
    chosen_sku: str,
    chosen_sku_idx: int = None,
    show_all_rows: bool = False
):
    def show(df, cols):
        cols = [c for c in cols if c in df.columns]
        display(df[cols])

    if show_all_rows:
        pd.set_option("display.max_rows", None)
    else:
        pd.reset_option("display.max_rows")
    pd.set_option("display.max_colwidth", None)

    if chosen_sku_idx is not None:
        print(f"\n📌 Query SKU {chosen_sku} (#{chosen_sku_idx}):")
    else:
        print(f"\n📌 Query SKU {chosen_sku}:")

    show(query_row, ['sku','name','width','height'] + display_cols)

    print(f"\n✅ Positives (count: {len(positives)}):")
    show(positives, ['sku','name'] + display_cols)

    print(f"\n❌ Hard negatives (count: {len(hard_negatives)}):")
    show(hard_negatives, ['sku','name'] + display_cols)

    print(f"\n🚫 Simple negative (rejected) (count: {len(rejected)}):")
    show(rejected, ['sku','name'] + display_cols)

In [20]:
# Process sku matches

(
    positives,
    hard_negatives,
    rejected,
    query_row,
    fail_cols,
    size_cols,
    pattern_cols
) = process_sku_matches(
    chosen_sku       = CHOSEN_SKU,
    pattern_dict     = ALL_PATTERNS_DICT[CHOSEN_SKU],
    size_matches_df  = size_matches_df,
    chosen_subset_df = chosen_subset_df,
    column_to_match  = COLUMN_TO_MATCH,
    margin           = MARGIN
)


In [21]:
# Print matches

print_sku_matches(
    positives, hard_negatives, rejected,
    query_row,

    display_cols=fail_cols + size_cols,
    # display_cols=fail_cols + pattern_cols,

    chosen_sku=CHOSEN_SKU,
    show_all_rows = SHOW_ALL_ROWS,
)


📌 Query SKU 1871769771:


,sku,name,width,height
0,1871769771,"Карты МИРА и РОССИИ настенные политические,160х102 см, Комплект школьных карт для детей / 2024 г",160,102



✅ Positives (count: 9):


,sku,name,width_matched_cand,width_matched_query,width_delta,height_matched_cand,height_matched_query,height_delta
1641,1674648694,"Двухсторонняя политическая карта мира и России с новыми границами 160х100 см на отвесах в тубусе, настенная, для офиса, школы, дома, ""АГТ Геоцентр""",160,160,0.0000,100,102,0.019608
2716,1421565752,"Двухсторонняя политическая карта мира и России 158х107 см настенная с новыми границами в тубусе ""АГТ Геоцентр""",158,160,0.0125,107,102,0.049020
2717,1421565737,"Двухсторонняя политико-административная карта мира и России 158х107 см на отвесах, настенная с новыми границами в тубусе ""АГТ Геоцентр""",158,160,0.0125,107,102,0.049020
2718,1421559792,"Двухсторонняя политико-административная карта мира и России 158х107 см на отвесах, настенная с новыми границами в тубусе ""АГТ Геоцентр""",158,160,0.0125,107,102,0.049020
2719,1421500089,"Двухсторонняя политическая карта мира и России 158х107 см настенная с новыми границами в тубусе ""АГТ Геоцентр""",158,160,0.0125,107,102,0.049020
4837,600779875,"Двухсторонняя политическая карта мира и России 160х100 см на отвесах в тубусе, настенная, для офиса, школы, дома, ""АГТ Геоцентр""",160,160,0.0000,100,102,0.019608
4838,600778837,"Двухсторонняя политическая карта мира и России 160х100 см с новыми границами в тубусе, настенная, для офиса, школы, дома, ""АГТ Геоцентр""",160,160,0.0000,100,102,0.019608
5345,467420540,"Двухсторонняя политическая карта мира и России с новыми границами 160х100 см на отвесах в тубусе, настенная, для офиса, школы, дома, ""АГТ Геоцентр""",160,160,0.0000,100,102,0.019608
5346,467396304,"Двухсторонняя политическая карта мира и России с новыми границами 160х100 см в тубусе, настенная, для офиса, школы, дома, ""АГТ Геоцентр""",160,160,0.0000,100,102,0.019608



❌ Hard negatives (count: 27):


,sku,name,fail_regex,fail_size,width_matched_cand,width_matched_query,width_delta,height_matched_cand,height_matched_query,height_delta
1775,1642067085,"Мир и Россия в ретро стиле политическая двухсторонняя карта 120х80 см настенная с новыми границами в тубусе ""АГТ Геоцентр""",True,False,120,160,0.25000,80,102,0.215686
2301,1540069061,Мир и Россия в ретро стиле политическая двухсторонняя карта на отвесах 120х80 см настенная с новыми границами в тубусе,True,False,120,160,0.25000,80,102,0.215686
2302,1540065800,"Мир и Россия в ретро стиле политическая двухсторонняя карта 120х80 см настенная с новыми границами в тубусе ""АГТ Геоцентр""",True,False,120,160,0.25000,80,102,0.215686
2309,1538722406,Мир и Россия в ретро стиле политическая двухсторонняя карта на отвесах 120х80 см настенная с новыми границами в тубусе,True,False,120,160,0.25000,80,102,0.215686
2311,1536520050,"Мир и Россия в ретро стиле политическая двухсторонняя карта 120х80 см настенная с новыми границами в тубусе ""АГТ Геоцентр""",True,False,120,160,0.25000,80,102,0.215686
3918,846260799,Компл. Карта настенная в тубусе 2 шт. Мир Политический + Россия Физическая. 101х69 см. ЛАМ ГЕОДОМ,True,False,101,160,0.36875,69,102,0.323529
5261,494562010,"Двусторонняя Карта Мира и России политическая, 70 х 100 см. Складная, школьная карта. Выпуск 2024",True,False,100,160,0.37500,70,102,0.313725
233,1873027006,"Настенная двусторонняя карта мира и России на рейках, 101х69 см",False,True,101,160,0.36875,69,102,0.323529
886,1743520965,"Двухсторонняя настенная карта Российской Федерации общегеографическая, карта мира политическая",False,True,96,160,0.40000,65,102,0.362745
1754,1645939094,Географическая карта,False,True,87,160,0.45625,58,102,0.431373



🚫 Simple negative (rejected) (count: 5525):


,sku,name
1,1679550303,"Схема линий скоростного транспорта Москвы (Метро, МЦК, МЦД) 0,5*0,7 м, ламинированная"
2,1200553001,"Политическая карта МИРА 160х109 см, Карта мира настенная, подарок для школьника"
3,922231521,"Политическая карта МИРА настенная, 100х70см, школьная географическая карта мира, Выпуск 2024 год, В ТУБУСЕ"
4,922230517,"Политическая карта МИРА настенная, 160х102см, карта мира по географии, Выпуск 2024 год, В ТУБУСЕ"
5,922230183,"Карта России настенная, 102х160 см, карта России по географии, Выпуск 2024 год, В ТУБУСЕ"
...,...,...
5557,166584090,"Карта настенная Мир. Политическая карта с флагами, М-1:30 млн., размер 122х79 см, ламинированная"
5558,166451882,"Карта мира, магнитно-маркерная, 60х40 см, комплект цветных маркеров со стирателем"
5559,154409524,Карта звездного неба космоса настенная светящаяся для детей в подарок Gagarin Map
5560,147896031,Скретч карта мира географическая Dark Edition (80х60см) в тубусе


In [22]:
# View particular sku in a particular subset dataframe

SKU_TO_VIEW = 1333611366

pd.set_option('display.max_columns', None)

full_df[full_df.sku == SKU_TO_VIEW][['sku', 'name', 'description']]

cols_to_drop = ['sku', 'name'] + fail_cols + size_cols + pattern_cols

# size_matches_df[size_matches_df.sku == SKU_TO_VIEW]

hard_negatives[
    hard_negatives.sku == SKU_TO_VIEW

    # (hard_negatives.fail_regex)
    # (~hard_negatives.fail_size) & (hard_negatives.fail_regex)
    # hard_negatives['N: окрестн']
    ][
        cols_to_drop    
    ]

# positives[positives.sku == SKU_TO_VIEW][
#         ['sku','name',
#         # 'width_matched_cand', 'height_matched_cand', 'width_delta','height_delta',
#         ] + size_cols + pattern_cols
# ]

# rejected[rejected.sku == SKU_TO_VIEW][['sku', 'name']]

,sku,name,fail_regex,fail_size,width_matched_cand,width_matched_query,width_delta,height_matched_cand,height_matched_query,height_delta,P: мира,P: политическ,P: россии,P: настенн,"P: комплект школьных карт|карта двух?сторонняя|двух?сторонняя(?:\s+\w+){0,5}\s+карта|\b(?:[2-9]|\d{2,})\s*шт(?:\.|ук|уки)?\b",N: ретро|старинн,N: физическ,N: фотообои,N: английск,N: фальцован|складн,N: полушар,N: скретч
2878,1333611366,"Двусторонняя карта мира+России, интерактивная, в тубусе",False,True,101,160,0.36875,69,102,0.323529,True,True,True,True,True,False,False,False,False,False,False,False


In [23]:
# # Check price of query vs positives
# %matplotlib inline
# import matplotlib.pyplot as plt

# # Display query SKU information
# display(query_row[['sku', 'final_price']])

# # Plot histogram of positive SKUs' prices
# ax = positives['final_price'].plot(kind='hist', bins=30, alpha=0.7, title='Price Distribution of Positive Matches')
# # Add a vertical line for the query SKU's price
# ax.axvline(query_row['final_price'].values[0], color='red', linestyle='dashed', linewidth=2, label='Query Price')

# # Add legend and labels
# ax.set_xlabel('Final Price')
# ax.set_ylabel('Count')
# ax.legend()
# plt.show()


### Pairwise dataset

In [24]:
# make_pairwise_for_query

from typing import Optional
import pandas as pd


def make_pairwise_for_query(
    query_row: pd.DataFrame,
    positives: pd.DataFrame,
    hard_negatives: pd.DataFrame,
    cols_to_drop: list,
    rejected: Optional[pd.DataFrame] = None,
) -> pd.DataFrame:
    """
    Builds a pairwise DataFrame for one query using vectorized pandas ops:
      - query columns get suffix '_query'
      - candidate columns get suffix '_candidate'
      - only a 'label' column beyond those
    Drops any columns listed in cols_to_drop from both sides.
    Optionally appends `rejected` candidates as soft-negative pairs with label=0.5.
    """
    # 1) prepare query side
    q = (
        query_row
        .drop(columns=cols_to_drop, errors='ignore')
        .iloc[[0]]                             # keep as single-row DF
        .rename(columns=lambda c: f"{c}_query")
        .assign(_tmpkey=1)
    )

    # 2) prepare positives
    pos = (
        positives
        .drop(columns=cols_to_drop, errors='ignore')
        .rename(columns=lambda c: f"{c}_candidate")
        .assign(_tmpkey=1)
    )
    pos_pairs = (
        pd.merge(q, pos, on="_tmpkey")
        .drop(columns=["_tmpkey"])
        .assign(label=1)
    )

    # 3) prepare hard_negatives
    neg = (
        hard_negatives
        .drop(columns=cols_to_drop, errors='ignore')
        .rename(columns=lambda c: f"{c}_candidate")
        .assign(_tmpkey=1)
    )
    neg_pairs = (
        pd.merge(q, neg, on="_tmpkey")
        .drop(columns=["_tmpkey"])
        .assign(label=0)
    )

    # 4) optionally prepare rejected (soft negatives)
    pairs = [pos_pairs, neg_pairs]
    if rejected is not None:
        rej = (
            rejected
            .drop(columns=cols_to_drop, errors='ignore')
            .rename(columns=lambda c: f"{c}_candidate")
            .assign(_tmpkey=1)
        )
        rej_pairs = (
            pd.merge(q, rej, on="_tmpkey")
            .drop(columns=["_tmpkey"])
            .assign(label=0.5)
        )
        pairs.append(rej_pairs)

    # 5) concatenate and return
    return pd.concat(pairs, ignore_index=True)

In [25]:
import ast
from tqdm import tqdm

# INCLUDE_SOFT_NEGATIVES = False
INCLUDE_SOFT_NEGATIVES = True

# MAX_SKU = 2  # Limit the number of SKUs to process
MAX_SKU = None  # Uncomment to process all SKUs

group_rows = []
for i, (query_sku, pattern_dict) in enumerate(tqdm(ALL_PATTERNS_DICT.items(), desc="Building group-indexed mapping")):
    if MAX_SKU and i >= MAX_SKU:
        break
    (
        positives,
        hard_negatives,
        rejected,
        query_row,
        fail_cols,
        size_cols,
        pattern_cols
    ) = process_sku_matches(
        chosen_sku       = query_sku,
        pattern_dict     = ALL_PATTERNS_DICT[query_sku],
        size_matches_df  = size_matches_df,
        chosen_subset_df = chosen_subset_df,
        column_to_match  = COLUMN_TO_MATCH,
        margin           = MARGIN
    )
    sku_pos = positives['sku'].tolist()
    sku_hard_neg = hard_negatives['sku'].tolist()
    if INCLUDE_SOFT_NEGATIVES:
        sku_soft_neg = rejected['sku'].tolist() if 'sku' in rejected else []
    else:
        sku_soft_neg = []
    group_rows.append({
        'sku_query': query_sku,
        'sku_pos': sku_pos,
        'sku_hard_neg': sku_hard_neg,
        'sku_soft_neg': sku_soft_neg,
    })

pairwise_mapping_df = pd.DataFrame(group_rows)

Building group-indexed mapping:   0%|          | 0/20 [00:00<?, ?it/s]

Building group-indexed mapping:  35%|███▌      | 7/20 [00:15<00:31,  2.44s/it]/tmp/ipykernel_57570/69008786.py:103: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  pd.concat([pattern_negs, neg_by_size])
Building group-indexed mapping:  45%|████▌     | 9/20 [00:16<00:14,  1.36s/it]/tmp/ipykernel_57570/69008786.py:103: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  pd.concat([pattern_negs, neg_by_size])
Building group-indexed mapping: 100%|██████████| 20/20 [00:30<00:00,  1.53s/it]


In [26]:
pairwise_mapping_df.head(1)

,sku_query,sku_pos,sku_hard_neg,sku_soft_neg
0,1871769771,"[1674648694, 1421565752, 1421565737, 1421559792, 1421500089, 600779875, 600778837, 467420540, 467396304]","[1642067085, 1540069061, 1540065800, 1538722406, 1536520050, 846260799, 494562010, 1873027006, 1743520965, 1645939094, 1593651486, 1573142945, 1573135817, 1573124945, 1418084594, 1333611366, 1296084931, 1252814265, 1192132059, 1192130778, 1192129356, 1166886051, 721999402, 601557370, 601557360, 497250136, 497243585]","[1679550303, 1200553001, 922231521, 922230517, 922230183, 922229770, 824158517, 1957134593, 1953623209, 1952128676, 1951295556, 1951280032, 1951279668, 1951276889, 1951262659, 1951262580, 1951262465, 1951262178, 1951262172, 1951262102, 1951262055, 1951261861, 1951261833, 1951261830, 1951261757, 1951261645, 1951261582, 1951261473, 1951261233, 1951259611, 1949798600, 1947596544, 1947327095, 1947160810, 1946044250, 1946042228, 1945093340, 1944921158, 1943156611, 1943155878, 1943154897, 1942833525, 1942833347, 1942816308, 1942105445, 1941901506, 1941834406, 1941702940, 1941587759, 1941587329, 1941548057, 1941515384, 1941414130, 1940853996, 1940495550, 1939700884, 1939662161, 1939660050, 1939659805, 1939659746, 1939659158, 1939659129, 1939656095, 1936547864, 1934870477, 1934863722, 1934561685, 1933648712, 1933545623, 1930774721, 1930676308, 1929800349, 1928993402, 1928986788, 1928982576, 1928918258, 1925773771, 1925705818, 1925684706, 1925266298, 1925266291, 1925266248, 1925264606, 1925264566, 1925264539, 1925257686, 1925257416, 1925253038, 1922956339, 1921284669, 1919437475, 1919282160, 1919237614, 1918977749, 1917810905, 1917803640, 1917177150, 1917171467, 1916509957, 1916509925, ...]"


In [27]:
# Save to Parquet
from joblib import hash
from pathlib import Path

pattern_dict_hash = hash(ALL_PATTERNS_DICT)
group_file_name = (
    'regex-pairwise-groups'
    f'_num-queries={len(pairwise_mapping_df.sku_query.unique())}'
    f'_patterns-dict-hash={pattern_dict_hash}'
    '.parquet'
)
group_file_path = (
    Path('../clip-siamese') / 'data' / 'tables_OZ_geo_5500' /
    'processed' / 'regex-pairwise-groups' / group_file_name
)
group_file_path.parent.mkdir(parents=True, exist_ok=True)
pairwise_mapping_df.to_parquet(group_file_path, index=False)
# Print file path without './clip-siamese/data/' part
prefix = Path('../clip-siamese') / 'data'
try:
    rel_path = group_file_path.relative_to(prefix)
except ValueError:
    rel_path = group_file_path
print(f'Group-indexed mapping saved to:\n{rel_path}')

Group-indexed mapping saved to:
tables_OZ_geo_5500/processed/regex-pairwise-groups/regex-pairwise-groups_num-queries=20_patterns-dict-hash=a6223255f273e52a893ba7235e3c19b3.parquet
